<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://raw.githubusercontent.com/Unidata/MetPy/master/src/metpy/plots/_static/unidata_150x150.png" alt="Unidata Logo" style="height: 98px;">
</div>

<h1>Working with Surface Observations in Siphon and MetPy</h1>
<h3>Unidata Python Workshop</h3>

<div style="clear:both"></div>
</div>

<hr style="height:2px;">

<div style="float:right; width:250 px"><img src="http://weather-geek.net/images/metar_what.png" alt="METAR" style="height: 200px;"></div>

### Questions
1. What's the best way to get surface station data from a THREDDS data server?
1. What's the best way to make a station plot of data?
1. How can I request a time series of data for a single station?

### Objectives
1. <a href="#metar">Getting METARs from THREDDS</a>
1. <a href="#parse">Parse the Data</a>
1. <a href="#stationplot">Making a Station Plot</a>
1. <a href="#timeseries">Time Series Request and Plot</a>

<a name="metar">Getting METARs from THREDDS</a>
## 1. Getting METARs from THREDDS

We can get the current METARS from the THREDDS test server (it's not yet available on the main thredds.ucar.edu). Head over to <a href="https://thredds-test.unidata.ucar.edu/">https://thredds-test.unidata.ucar.edu/</a> and navigate to the NOAAport products and find the files.

In [ ]:
from siphon.catalog import TDSCatalog

In [ ]:
cat = TDSCatalog('https://thredds-test.unidata.ucar.edu/thredds/catalog/noaaport/text/metar/catalog.xml')

In [ ]:
ds = cat.datasets[0]

In [ ]:
# Ideally we would use remote_open, but because of a soon to be fixed bug we
# have to download locally and open that way.
# fobj = ds.remote_open()

In [ ]:
ds.download(f'../../../data/{ds.name}')

Open the file and take a look - aren't you glad MetPy can parse this for you?

<a href="#top">Top</a>
<hr style="height:2px;">

<a name="parse"></a>
## 2. Parse the Data
MetPy can parse the METAR data for us into a dataframe that the declarative plotting inface can work with nicely.

In [ ]:
from metpy.io import parse_metar_file

In [ ]:
df = parse_metar_file(f'../../../data/{ds.name}')

In [ ]:
df.head()

Let's take a bit of a closer look at the columns that are available to use as well.

In [ ]:
list(df.columns)

<a href="#top">Top</a>
<hr style="height:2px;">

<a name="stationplot"></a>
## 3. Making a Station Plot

In [ ]:
from datetime import datetime, timedelta

import cartopy.crs as ccrs

from metpy.plots.declarative import *

In [ ]:
obs = PlotObs()
obs.data = df
obs.time = datetime.utcnow()
obs.level = None
obs.fields = ['air_temperature']
obs.locations = ['NW']
obs.colors = ['tab:red']
obs.formats = [None]
obs.vector_field = ['eastward_wind', 'northward_wind']
obs.reduce_points = 0.5

panel = MapPanel()
panel.area = 'ma'
panel.projection = ccrs.PlateCarree()
panel.layers = ['coastline', 'borders', 'states']

panel.plots = [obs]

pc = PanelContainer()
pc.size = (10, 10)
pc.panels = [panel]

pc.show()

<div class="alert alert-success">
    <b>EXERCISE</b>:

Make a surface plot that plots:

 <ul>
     <li>Temperature in red, NW plot area</li>
     <li>Dewpoint in green, SW plot area</li>
     <li>Altimeter setting in black, NE plot area</li>
     <li>Sky coverage in black, Center plot area</li>
     <li>Set the title to something relevant</li>
</ul>

<b>BONUS: Format the altimeter setting in the "traditional" way (multiply by 10 and show only the last three digits of the integer value). i.e. 1014.56 becomes 145.</b>
</div>

In [ ]:
# Make the observation plot

# Make the map panel

# Make the panel container

# Show the plot


<div class="alert alert-info">
    <b>SOLUTION</b>
</div>

In [ ]:
# %load solutions/dec_basic_sfc_plot.py

<a href="#top">Top</a>
<hr style="height:2px;">

<a name="timeseries"></a>
## 4. Time Series Request and Plot
Let's say we want the past days worth of data for a lat/lon point for the variables mean sea level pressure, air temperature, wind direction, and wind speed. We do not have time series plots in a declarative way yet, but these plots are relatively straight forward to create and it's another way we can query THREDDS to get data.

In [ ]:
metar_cat_url = ('http://thredds.ucar.edu/thredds/catalog/'
                 'irma/metar/catalog.xml?dataset=irma/metar/Metar_Station_Data_-_Irma_fc.cdmr')

catalog = TDSCatalog(metar_cat_url)

metar_dataset = catalog.datasets['Feature Collection']

ncss = metar_dataset.subset()

In [ ]:
# Define the time range we are interested in
end_time = datetime(2017, 9, 12, 0)
start_time = end_time - timedelta(days=2)

# Build the query
query = ncss.query()
query.lonlat_point(-80.25, 25.8)
query.time_range(start_time, end_time)
query.variables('altimeter_setting', 'temperature', 'dewpoint',
                'wind_direction', 'wind_speed')
query.accept('csv')

In [ ]:
# Get the data
data = ncss.get_data(query)

In [ ]:
# Make sure we got what we asked for
print(list(data.keys()))

### Cleanup the Data

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.head()

In [ ]:
# Parse the date time stamps
df['time'] = pd.to_datetime(df['time'].str.decode('utf-8'), infer_datetime_format=True)

In [ ]:
df.head()

In [ ]:
# Station names are bytes, we need to convert them to strings
df['station'] = df['station'].str.decode('utf-8')

In [ ]:
df.head()

### Make a Time Series Plot

In [ ]:
ax = df.plot(x='time', y='wind_speed',
             title=f"{df['station'][0]} {df['time'][0]:%Y/%m/%d}",
             grid=True,
             figsize=(10, 6))

In [ ]:
from matplotlib.dates import DateFormatter, AutoDateLocator
# Improve on the default ticking
locator = AutoDateLocator()
hoursFmt = DateFormatter('%H')
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(hoursFmt)
fig = ax.get_figure()
fig

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
        <li>Pick a different location.</li>
        <li>Plot temperature and dewpoint together on the same plot.</li>
        <li>Use ax.set.xlabel and the corresponding y label to set sensible labels.</li>
    </ul>
</div>

In [ ]:
# Your code goes here


In [ ]:
# %load solutions/pd_time_series.py

<a href="#top">Top</a>
<hr style="height:2px;">